In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# Any results you write to the current directory are saved as output.

In [2]:
test = pd.read_csv("C:/Users/Public/Dataset/test.csv")
train = pd.read_csv("C:/Users/Public/Dataset/train.csv")

In [3]:
train.head()

,PassengerId,survived,Pclass,Name,Sex,Age,SibSp,Parch,Ticket,Fare,Cabin,Embarked
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,A/5 21171,7.2500,NaN,S
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,PC 17599,71.2833,C85,C
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,STON/O2. 3101282,7.9250,NaN,S
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,113803,53.1000,C123,S
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,373450,8.0500,NaN,S


In [4]:
train.isnull().sum()

PassengerId      0
survived         0
Pclass           0
Name             0
Sex              0
Age            177
SibSp            0
Parch            0
Ticket           0
Fare             0
Cabin          687
Embarked         2
dtype: int64

**Check the train and test set data and get some information from it**

In [5]:
test.isnull().sum()

PassengerId      0
Pclass           0
Name             0
Sex              0
Age             86
SibSp            0
Parch            0
Ticket           0
Fare             1
Cabin          327
Embarked         0
dtype: int64

In [6]:
train.describe(include=['O'])

,Name,Sex,Ticket,Cabin,Embarked
count,891,891,891,204,889
unique,891,2,681,147,3
top,"Braund, Mr. Owen Harris",male,347082,B96 B98,S
freq,1,577,7,4,644


**As we can see above, Ticket and Cabin are mostly shared, so we can actually drop these two features from the data set**

In [7]:
dataset = [train,test]
for data in dataset:
    data.drop(['Ticket','Cabin'], axis = 1, inplace = True)

**Age, Embarked and Fare have null values in train and test set, lets fill the null values with median value for Age, mode value for Embarked and Median value for Fare**

In [8]:
null_cols = ['Age','Embarked','Fare']
for data in dataset:
    data['Age'].fillna(data['Age'].median(), inplace = True)
    data['Embarked'].fillna(data['Embarked'].mode()[0], inplace = True)
    data['Fare'].fillna(data['Fare'].median(), inplace = True)

In [9]:
train.isnull().sum()

PassengerId    0
survived       0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

In [10]:
test.isnull().sum()

PassengerId    0
Pclass         0
Name           0
Sex            0
Age            0
SibSp          0
Parch          0
Fare           0
Embarked       0
dtype: int64

**Now, we have managed to get rid of Null values**

**Lets extract the titles(Mr,Mrs,Ms,..) from the Name column and create a new attribute called Title**

In [11]:
for data in dataset:
    data['Title'] = data['Name'].str.split(',',expand = True)[1].str.split('.',expand = True)[0]

In [12]:
train['Title'].value_counts()

 Mr              517
 Miss            182
 Mrs             125
 Master           40
 Dr                7
 Rev               6
 Mlle              2
 Major             2
 Col               2
 the Countess      1
 Capt              1
 Ms                1
 Sir               1
 Lady              1
 Mme               1
 Don               1
 Jonkheer          1
Name: Title, dtype: int64

In [13]:
test['Title'].value_counts()

 Mr        240
 Miss       78
 Mrs        72
 Master     21
 Col         2
 Rev         2
 Ms          1
 Dr          1
 Dona        1
Name: Title, dtype: int64

In [14]:
train_title_counts = train['Title'].value_counts() < 10
test_title_counts = test['Title'].value_counts() < 10

**Apart from Mr, Mrs, Ms there are some special titles like, Rev, Major, col, capt.. Lets replace all those special titles with Misc**

In [15]:
Misc_count = 10


In [16]:
train['Title'] = train['Title'].apply(lambda x: 'Misc' if train_title_counts.loc[x] == True else x)
test['Title'] = test['Title'].apply(lambda x: 'Misc' if test_title_counts.loc[x] == True else x)

**PassengerId, Name and Cabin will not be of much use in determining the survival category, so we will drop them from our train and test data set**

In [17]:
train['Title'].value_counts()

 Mr        517
 Miss      182
 Mrs       125
 Master     40
Misc        27
Name: Title, dtype: int64

In [18]:
test['Title'].value_counts()

 Mr        240
 Miss       78
 Mrs        72
 Master     21
Misc         7
Name: Title, dtype: int64

In [19]:
train.columns

Index(['PassengerId', 'survived', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp',
       'Parch', 'Fare', 'Embarked', 'Title'],
      dtype='object')

In [20]:
test.columns

Index(['PassengerId', 'Pclass', 'Name', 'Sex', 'Age', 'SibSp', 'Parch', 'Fare',
       'Embarked', 'Title'],
      dtype='object')

**We can create two new features from SibSp and Parch features, namely: Family_size and Is_alone. So lets create them**

In [21]:
for data in dataset:
    data['Family_size'] = data['SibSp'] + data['Parch'] + 1
    data['Is_alone'] = 1
    data['Is_alone'].loc[data['Family_size'] > 1] = 0

C:\Users\dixit\AppData\Local\Temp\ipykernel_17224\1293480760.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Is_alone'].loc[data['Family_size'] > 1] = 0
C:\Users\dixit\AppData\Local\Temp\ipykernel_17224\1293480760.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  data['Is_alone'].loc[data['Family_size'] > 1] = 0


In [22]:
train['Is_alone'].value_counts()

1    537
0    354
Name: Is_alone, dtype: int64

In [23]:
test['Is_alone'].value_counts()

1    253
0    165
Name: Is_alone, dtype: int64

In [24]:
train['Family_size'].value_counts()

1     537
2     161
3     102
4      29
6      22
5      15
7      12
11      7
8       6
Name: Family_size, dtype: int64

In [25]:
test['Family_size'].value_counts()

1     253
2      74
3      57
4      14
5       7
7       4
11      4
6       3
8       2
Name: Family_size, dtype: int64

**We can now convert the Fare and Age**

In [26]:
for data in dataset:
    data['Fare_bin'] = pd.qcut(data['Fare'], 4)
    data['Age_bin'] = pd.cut(data['Age'].astype(int), 5 )

In [27]:
train.head()

,PassengerId,survived,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Family_size,Is_alone,Fare_bin,Age_bin
0,1,0,3,"Braund, Mr. Owen Harris",male,22.0,1,0,7.2500,S,Mr,2,0,"(-0.001, 7.91]","(16.0, 32.0]"
1,2,1,1,"Cumings, Mrs. John Bradley (Florence Briggs Th...",female,38.0,1,0,71.2833,C,Mrs,2,0,"(31.0, 512.329]","(32.0, 48.0]"
2,3,1,3,"Heikkinen, Miss. Laina",female,26.0,0,0,7.9250,S,Miss,1,1,"(7.91, 14.454]","(16.0, 32.0]"
3,4,1,1,"Futrelle, Mrs. Jacques Heath (Lily May Peel)",female,35.0,1,0,53.1000,S,Mrs,2,0,"(31.0, 512.329]","(32.0, 48.0]"
4,5,0,3,"Allen, Mr. William Henry",male,35.0,0,0,8.0500,S,Mr,1,1,"(7.91, 14.454]","(32.0, 48.0]"


In [28]:
test.head()

,PassengerId,Pclass,Name,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Family_size,Is_alone,Fare_bin,Age_bin
0,892,3,"Kelly, Mr. James",male,34.5,0,0,7.8292,Q,Mr,1,1,"(-0.001, 7.896]","(30.4, 45.6]"
1,893,3,"Wilkes, Mrs. James (Ellen Needs)",female,47.0,1,0,7.0000,S,Mrs,2,0,"(-0.001, 7.896]","(45.6, 60.8]"
2,894,2,"Myles, Mr. Thomas Francis",male,62.0,0,0,9.6875,Q,Mr,1,1,"(7.896, 14.454]","(60.8, 76.0]"
3,895,3,"Wirz, Mr. Albert",male,27.0,0,0,8.6625,S,Mr,1,1,"(7.896, 14.454]","(15.2, 30.4]"
4,896,3,"Hirvonen, Mrs. Alexander (Helga E Lindqvist)",female,22.0,1,1,12.2875,S,Mrs,3,0,"(7.896, 14.454]","(15.2, 30.4]"


In [29]:
train.drop(['PassengerId','Name'], axis = 1, inplace = True)
test.drop(['Name'], axis = 1, inplace = True)

In [30]:
train.head()

,survived,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Family_size,Is_alone,Fare_bin,Age_bin
0,0,3,male,22.0,1,0,7.2500,S,Mr,2,0,"(-0.001, 7.91]","(16.0, 32.0]"
1,1,1,female,38.0,1,0,71.2833,C,Mrs,2,0,"(31.0, 512.329]","(32.0, 48.0]"
2,1,3,female,26.0,0,0,7.9250,S,Miss,1,1,"(7.91, 14.454]","(16.0, 32.0]"
3,1,1,female,35.0,1,0,53.1000,S,Mrs,2,0,"(31.0, 512.329]","(32.0, 48.0]"
4,0,3,male,35.0,0,0,8.0500,S,Mr,1,1,"(7.91, 14.454]","(32.0, 48.0]"


In [31]:
test.head()

,PassengerId,Pclass,Sex,Age,SibSp,Parch,Fare,Embarked,Title,Family_size,Is_alone,Fare_bin,Age_bin
0,892,3,male,34.5,0,0,7.8292,Q,Mr,1,1,"(-0.001, 7.896]","(30.4, 45.6]"
1,893,3,female,47.0,1,0,7.0000,S,Mrs,2,0,"(-0.001, 7.896]","(45.6, 60.8]"
2,894,2,male,62.0,0,0,9.6875,Q,Mr,1,1,"(7.896, 14.454]","(60.8, 76.0]"
3,895,3,male,27.0,0,0,8.6625,S,Mr,1,1,"(7.896, 14.454]","(15.2, 30.4]"
4,896,3,female,22.0,1,1,12.2875,S,Mrs,3,0,"(7.896, 14.454]","(15.2, 30.4]"


**We can drop SibSp and Parch attributes as well as we have derived a new attribute called as Family_size, these are called as redundant attributes and may bring down the score of the model. we can also drop Age and Fare attributes and we have converted them into ordinal attributes Age_bin and Fare_bin respectively**

In [32]:
#for data in dataset:
 #   data.drop(['SibSp','Parch','Age','Fare'], axis = 1, inplace = True)
    
for data in dataset:
    data.drop(['Age','Fare'], axis = 1, inplace = True)

In [33]:
train.head()

,survived,Pclass,Sex,SibSp,Parch,Embarked,Title,Family_size,Is_alone,Fare_bin,Age_bin
0,0,3,male,1,0,S,Mr,2,0,"(-0.001, 7.91]","(16.0, 32.0]"
1,1,1,female,1,0,C,Mrs,2,0,"(31.0, 512.329]","(32.0, 48.0]"
2,1,3,female,0,0,S,Miss,1,1,"(7.91, 14.454]","(16.0, 32.0]"
3,1,1,female,1,0,S,Mrs,2,0,"(31.0, 512.329]","(32.0, 48.0]"
4,0,3,male,0,0,S,Mr,1,1,"(7.91, 14.454]","(32.0, 48.0]"


In [34]:
test.head()

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked,Title,Family_size,Is_alone,Fare_bin,Age_bin
0,892,3,male,0,0,Q,Mr,1,1,"(-0.001, 7.896]","(30.4, 45.6]"
1,893,3,female,1,0,S,Mrs,2,0,"(-0.001, 7.896]","(45.6, 60.8]"
2,894,2,male,0,0,Q,Mr,1,1,"(7.896, 14.454]","(60.8, 76.0]"
3,895,3,male,0,0,S,Mr,1,1,"(7.896, 14.454]","(15.2, 30.4]"
4,896,3,female,1,1,S,Mrs,3,0,"(7.896, 14.454]","(15.2, 30.4]"


In [35]:
cat_attr = ['Sex','Title','Embarked','Fare_bin','Age_bin']

In [36]:
from sklearn.preprocessing import LabelEncoder
encoder = LabelEncoder()
for data in dataset:
    for attr in cat_attr:
        data[attr] = encoder.fit_transform(data[attr])
    

In [37]:
train.head()

,survived,Pclass,Sex,SibSp,Parch,Embarked,Title,Family_size,Is_alone,Fare_bin,Age_bin
0,0,3,1,1,0,2,2,2,0,0,1
1,1,1,0,1,0,0,3,2,0,3,2
2,1,3,0,0,0,2,1,1,1,1,1
3,1,1,0,1,0,2,3,2,0,3,2
4,0,3,1,0,0,2,2,1,1,1,2


In [38]:
test.head()

,PassengerId,Pclass,Sex,SibSp,Parch,Embarked,Title,Family_size,Is_alone,Fare_bin,Age_bin
0,892,3,1,0,0,1,2,1,1,0,2
1,893,3,0,1,0,2,3,2,0,0,3
2,894,2,1,0,0,1,2,1,1,1,4
3,895,3,1,0,0,2,2,1,1,1,1
4,896,3,0,1,1,2,3,3,0,1,1


**Lets find the correlation of Survival with rest of the attributes(independent variables)**

In [39]:
attr = ['Pclass','Sex','Embarked','Family_size','Is_alone','Fare_bin','Age_bin','SibSp','Parch','Title']
for col in attr:
    print(train[['Survived',col]].groupby(col).mean())
    print()

KeyError: "['Survived'] not in index"

In [ ]:
train.corr()

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
X_train,X_val,y_train,y_val = train_test_split(train.drop(['Survived','Family_size'],axis = 1),train['Survived'],test_size = 0.2, shuffle = True,stratify = train['Survived'],random_state = 42)

In [ ]:
X_train.shape

In [ ]:
X_train.columns

In [ ]:
X_val.shape

In [ ]:
X_val.columns

**Let us try out different classification algorithms and get the accuracy**

In [ ]:
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.linear_model import LogisticRegression
from xgboost import XGBClassifier

In [ ]:
DTC = DecisionTreeClassifier()
RFC = RandomForestClassifier()
NBC = GaussianNB()
SVMC = SVC()
LRC = LogisticRegression()
XGBC = XGBClassifier()

In [ ]:
MLAs = [DTC,RFC,NBC,SVMC,LRC,XGBC]

In [ ]:
train.columns

In [ ]:
test.columns

In [ ]:
X_train.columns

In [ ]:
X_val.columns

In [ ]:
test.columns

In [ ]:
for algo in MLAs:
    algo.fit(X_train,y_train)
    print("Algorithm: ",algo)
    print("Training accuracy: ", algo.score(X_train,y_train))
    print("validation accuracy: ", algo.score(X_val,y_val))
    print("**************************************\n")

In [ ]:
RFC = RandomForestClassifier(n_estimators = 300, criterion = 'gini', max_depth = 4, random_state = 42)

In [ ]:
RFC.fit(X_train,y_train)
print("Training accuracy: ", RFC.score(X_train,y_train))
print("validation accuracy: ", RFC.score(X_val,y_val))

In [ ]:
test.head()

In [ ]:
X_train.head()

**As we are not using Family_size feature in our training step, we will drop this attribute from our test set as well**

In [ ]:
test.drop(['Family_size'], axis = 1, inplace = True)

**Lets compare the attributes used for training the model and those present in the test set **

In [ ]:
X_train.columns

In [ ]:
test.columns

In [ ]:
predictions = RFC.predict(test.drop(['PassengerId','Survived'],axis = 1))

In [ ]:
len(predictions)

In [ ]:
test.shape

In [ ]:
submit['PassengerId'] = test['PassengerId']

In [ ]:
submit['Survived'] = predictions

In [ ]:
test.head()

In [ ]:
test['Survived'] = predictions

In [ ]:
test.head()

In [ ]:
del submit

In [ ]:
submit = test[['PassengerId','Survived']]

In [ ]:
submit.shape

In [ ]:
submit['Survived'].value_counts(normalize=True)

In [ ]:
submit.sample(10)

In [ ]:
submit.to_csv("../working/submit.csv", index=False)